In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('all_annotations.txt', sep="\t", header=None, names=["chr","source","type","start","stop","s1","s2","s3","info"])

In [3]:
df.drop(df[df['type'] == "region"].index, inplace = True)

In [34]:
df.type.unique()

array(['gene', 'mRNA', 'exon', 'CDS', 'snRNA', 'pseudogene', 'cDNA_match',
       'tRNA', 'lnc_RNA', 'transcript', 'snoRNA', 'rRNA',
       'primary_transcript', 'miRNA', 'sequence_feature', 'tandem_repeat',
       'promoter'], dtype=object)

In [4]:
loci = pd.read_csv('top_goodhe_01', header=None)

In [7]:
new_df=pd.DataFrame()
for index, row in loci.iterrows():
    ch = row[0].split("_")[:2]
    ch = "_".join(ch)
    pos = int(row[0].split("_")[2])
    temp=df[(df['chr']==ch) & (df['start']<=pos) & (df['stop']>=pos)]
    if len(temp) == 0:
        #print("not in annotation")
        new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "not_annot", "gene" : "NA"},  
                        ignore_index = True) 
    if len(temp) == 1:
        print("only one")
        print(temp['type'])
    if len(temp) > 1:
        #print("more than 1")
        #checking for gene overlap
        ids=[]
        for i in range(len(temp)):
            t=temp["info"].iloc[i].split(";")
            res = list(filter(lambda x: "LOC" in x, t))
            if len(res) == 0:
                res = list(filter(lambda x: "GeneID:" in x, t))
                if len(res) != 0:
                    ID = res[0].split("GeneID:",1)[1]
                    ids.append(ID)
            else:
                ID =res[0].split("LOC",1)[1]
                ids.append(ID)
        uni=len(Counter(ids))
        gene_name = ids[0]
        if uni > 1:
            #print("genes overlap")
            new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "genes_overlap", "gene": "NA"},  
                        ignore_index = True) 
        
        if uni == 1:
            if temp['type'].str.contains('gene').any():
                if temp['type'].str.contains('pseudogene').any():
                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "pseudogene", "gene":"NA" },ignore_index = True)
                else:
                    if temp['type'].str.contains('exon').any(): 
                        if temp['type'].str.contains('CDS').any():
                            #print("in exon")
                            new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "exon", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                        else:
                            direction=temp['s2'].iloc[0]
                            e_start=temp[temp["type"]=="exon"].iloc[0].start
                            e_stop=temp[temp["type"]=="exon"].iloc[0].stop
                            g_start=temp[temp["type"]=="gene"].iloc[0].start
                            g_stop=temp[temp["type"]=="gene"].iloc[0].stop
                            if direction == "+":
                                if e_start == g_start:
                                    #print('5prime1')
                                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "5'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                                elif e_stop == g_stop:
                                    #print("3prime1")
                                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "3'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                                else:
                                    print("ERROR1") #alternative UTR?
                                    #print(temp)
                                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "alternative UTR", "gene":"NA" },ignore_index = True) 
                            elif direction == "-":
                                if e_start == g_start:
                                    #print('3prime2')
                                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "3'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True) 
                                elif e_stop == g_stop:
                                    #print("5prime2")
                                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "5'UTR", "gene":f"LOC{ids[0]}" },ignore_index = True)
                                else:
                                    print("ERROR2")
                                    new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "alternative UTR", "gene":"NA" },ignore_index = True) 
                            else:
                                print("ERROR3")
                                #print(temp)

                    else:
                        #print("in intron")
                        new_df = new_df.append({'chr' : ch, 'pos' : pos, 'hol' : "intron", "gene":f"LOC{ids[0]}" },ignore_index = True) 

            else:
                print("not gene")
    

ERROR1
ERROR2
ERROR1
ERROR1
ERROR1
ERROR2
ERROR2
ERROR2
ERROR2
ERROR2
ERROR2
ERROR1
ERROR2
ERROR2
ERROR1
ERROR2
ERROR2
ERROR2
ERROR1
ERROR2
ERROR1
ERROR1
ERROR1
ERROR2
ERROR2
ERROR1
ERROR1
ERROR2
ERROR2
ERROR1
ERROR2
ERROR1
ERROR2
ERROR2
ERROR1


In [11]:
new_df

,chr,gene,hol,pos
0,NW_022145599.1,NA,not_annot,809765
1,NW_022145613.1,NA,not_annot,22684111
2,NW_022145600.1,NA,not_annot,36849648
3,NW_022145612.1,NA,not_annot,13121687
4,NW_022145595.1,NA,not_annot,20882520
...,...,...,...,...
6842,NW_022145597.1,NA,not_annot,27269202
6843,NW_022145597.1,NA,not_annot,25996126
6844,NW_022145596.1,NA,alternative UTR,12214610
6845,NW_022145595.1,NA,not_annot,3170334


In [10]:
new_df.pos = new_df.pos.astype(int)

In [12]:
new_df.to_csv("annotated_top_goodhe_01.csv",index=False)

In [23]:
len(new_df[new_df["hol"]=="pseudogene"])

2

In [24]:
new_df.hol.unique()

array(['not_annot', 'exon', "3'UTR", 'intron', 'genes_overlap',
       'alternative UTR', "5'UTR", 'pseudogene'], dtype=object)

## now let's see if there is any in enhancer, outside of intron

are intronic ones located in:
    1 ATAC
    2 lvar
    3 chip 
    4 lnc
    5 promoter
    6 TFBS
    

In [25]:
promoters = pd.read_csv('all_annotations.txt', sep="\t", header=None, names=["chr","source","type","start","stop","s1","s2","s3","info"])
promoters = promoters[promoters['type']=="gene"]

promoters.loc[(promoters["type"]=="gene") & (promoters["s2"]=="+"), "start"] -= 1000
promoters.loc[(promoters["type"]=="gene") & (promoters["s2"]=="-"), "stop"] += 1000

In [26]:
atac = pd.read_csv("atac_mapped_translatedto5.0.csv", header=None, names=["chr","start","stop"])
chip = pd.read_csv('AlxChip_mapped_translatedto5.0.txt', sep="\t", header=None, names=["chr","start","stop"])
lvar = pd.read_csv('lvar_spur_48_50_mapped_translatedto50.csv', header=None, names=["chr","start","stop"])
lnc = pd.read_csv("lncRNA_mapped_translatedto5.0.csv", header=1, names=["chr","start","stop"])

In [27]:
mis = new_df[new_df['hol']=="not_annot"]

In [28]:
enh_check=pd.DataFrame()
for index, row in mis.iterrows():
    ch = row.chr
    pos = row.pos
    temp=atac[(atac['chr']==ch) & (atac['start']<=pos) & (atac['stop']>=pos)]
    if len(temp) > 0:
        print("in atac")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "atac" },ignore_index = True) 
    temp=chip[(chip['chr']==ch) & (chip['start']<=pos) & (chip['stop']>=pos)]
    if len(temp) > 0:
        print("in chip")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "chip" },ignore_index = True)
    temp=lvar[(lvar['chr']==ch) & (lvar['start']<=pos) & (lvar['stop']>=pos)]
    if len(temp) > 0:
        print("in lvar")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lvar" },ignore_index = True)
    temp=lnc[(lnc['chr']==ch) & (lnc['start']<=pos) & (lnc['stop']>=pos)]
    if len(temp) > 0:
        print("in lnc")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lnc" },ignore_index = True)
    temp=promoters[(promoters['chr']==ch) & (promoters['start']<=pos) & (promoters['stop']>=pos)]
    if len(temp) > 0:
        print("in promoters")
        enh_check = enh_check.append({'chr' : ch, 'pos' : pos, 'hol' : "promoter" },ignore_index = True)

in promoters
in promoters
in lnc
in promoters
in promoters
in lnc
in lnc
in lvar
in lnc
in promoters
in promoters
in lnc
in lnc
in promoters
in lnc
in lnc
in promoters
in promoters
in lnc
in lnc
in promoters
in lnc
in promoters
in lnc
in promoters
in promoters
in lnc
in promoters
in promoters
in lnc
in lnc
in lnc
in lnc
in promoters
in lnc
in lnc
in lnc
in lnc
in promoters
in lnc
in promoters
in lnc
in lnc
in chip
in lnc
in lnc
in lnc
in lnc
in lvar
in lnc
in promoters
in promoters
in lnc
in lnc
in lnc
in promoters
in promoters
in lnc
in promoters
in lnc
in promoters
in promoters
in lnc
in lnc
in promoters
in promoters
in lnc
in promoters
in promoters
in lnc
in promoters
in promoters
in lnc
in lnc
in lnc
in promoters
in lvar
in lvar
in lnc
in promoters
in lnc
in promoters
in promoters
in lnc
in promoters
in promoters
in lnc
in lnc
in lnc
in promoters
in lnc
in lnc
in lnc
in promoters
in lnc
in lnc
in lnc
in promoters
in lvar
in promoters
in lnc
in lnc
in lnc
in lnc
in chip
in lnc
in pr

In [31]:
enh_check.hol.unique()

array(['promoter', 'lnc', 'lvar', 'chip'], dtype=object)

In [37]:
pot_en = enh_check[(enh_check["hol"]=="lvar") | (enh_check["hol"]=="chip")]

In [38]:
pot_en=pot_en.sort_values(by=["chr",'pos'])

In [40]:
len(pot_en)

19

In [ ]:
#reminder: chosen atac supp (or lvar or chip) too strict?

## now let's see if there is any in enhancer, inside of intron

In [41]:
mis = new_df[new_df['hol']=="intron"]

In [42]:
int_check=pd.DataFrame()
for index, row in mis.iterrows():
    ch = row.chr
    pos = row.pos
    temp=atac[(atac['chr']==ch) & (atac['start']<=pos) & (atac['stop']>=pos)]
    if len(temp) > 0:
        print("in atac")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "atac" },ignore_index = True) 
    temp=chip[(chip['chr']==ch) & (chip['start']<=pos) & (chip['stop']>=pos)]
    if len(temp) > 0:
        print("in chip")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "chip" },ignore_index = True)
    temp=lvar[(lvar['chr']==ch) & (lvar['start']<=pos) & (lvar['stop']>=pos)]
    if len(temp) > 0:
        print("in lvar")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lvar" },ignore_index = True)
    temp=lnc[(lnc['chr']==ch) & (lnc['start']<=pos) & (lnc['stop']>=pos)]
    if len(temp) > 0:
        print("in lnc")
        int_check = int_check.append({'chr' : ch, 'pos' : pos, 'hol' : "lnc" },ignore_index = True)

in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in chip
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in atac
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in chip
in lnc
in lnc
in chip
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in chip
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lvar
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lvar
in lvar
in lnc
in lnc
in lvar
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lvar
in atac
in lnc
in lnc
in lvar
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
in lnc
i

In [43]:
int_check.hol.unique()

array(['lnc', 'chip', 'atac', 'lvar'], dtype=object)

In [47]:
int_check[int_check["hol"]=="lvar"]

,chr,hol,pos
83,NW_022145604.1,lvar,8855042.0
96,NW_022145600.1,lvar,18662627.0
97,NW_022145612.1,lvar,47720769.0
100,NW_022145610.1,lvar,35316800.0
114,NW_022145595.1,lvar,16140373.0
118,NW_022145605.1,lvar,16695182.0
142,NW_022145612.1,lvar,14929793.0


In [48]:
pot_in = int_check[(int_check["hol"]=="lvar") | (int_check["hol"]=="chip") | (int_check["hol"]=="atac")]

In [50]:
pot_en=pot_en.sort_values(by=["chr",'pos'])

In [51]:
pot_en

,chr,hol,pos
77,NW_022145594.1,lvar,32414825.0
247,NW_022145597.1,chip,15623469.0
199,NW_022145597.1,chip,23728887.0
76,NW_022145598.1,lvar,11083748.0
339,NW_022145598.1,lvar,24095444.0
246,NW_022145600.1,lvar,1071667.0
43,NW_022145600.1,chip,21062821.0
224,NW_022145602.1,chip,7615785.0
140,NW_022145603.1,chip,5814450.0
104,NW_022145604.1,chip,11429281.0
